In [61]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [62]:
EMPTY = 0
PLAYER_X = 1
PLAYER_O = -1

In [63]:
memo = {}

In [64]:
def generate_win_patterns(n):
    """Generate win patterns for an n x n Tic-Tac-Toe board."""
    win_patterns = []

    # Rows and columns
    for i in range(n):
        win_patterns.append([(i, j) for j in range(n)])  # Row
        win_patterns.append([(j, i) for j in range(n)])  # Column

    # Diagonals
    win_patterns.append([(i, i) for i in range(n)])  # Main diagonal
    win_patterns.append([(i, n - 1 - i) for i in range(n)])  # Anti-diagonal

    return win_patterns

In [65]:
def check_win(board, win_patterns):
    """Checks if there is a winner on the board."""
    for pattern in win_patterns:
        values = [board[x][y] for x, y in pattern]
        if all(v == PLAYER_X for v in values):
            return PLAYER_X
        elif all(v == PLAYER_O for v in values):
            return PLAYER_O
    return None

In [66]:
def board_full(board):
    """Returns True if the board is full (no empty spaces)."""
    return all(board[i][j] != EMPTY for i in range(len(board)) for j in range(len(board)))


In [67]:
def evaluate_state(board, player, win_patterns, alpha=-float('inf'), beta=float('inf')):
    """Evaluates the current board state using alpha-beta pruning."""
    board_tuple = tuple(tuple(row) for row in board)  # Convert to tuple for memoization

    if board_tuple in memo:
        return memo[board_tuple]

    winner = check_win(board, win_patterns)
    if winner is not None:
        memo[board_tuple] = winner
        return winner

    if board_full(board):
        memo[board_tuple] = 0  # Draw
        return 0

    # Recursive case: Try all possible moves for the current player
    if player == PLAYER_X:  # Maximizing player (X)
        best_value = -float('inf')
        for i in range(len(board)):
            for j in range(len(board)):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_X
                    value = evaluate_state(board, PLAYER_O, win_patterns, alpha, beta)
                    best_value = max(best_value, value)
                    board[i][j] = EMPTY
                    alpha = max(alpha, best_value)
                    if beta <= alpha:  # Beta cut-off
                        break
        memo[board_tuple] = best_value
        return best_value
    else:  # Minimizing player (O)
        best_value = float('inf')
        for i in range(len(board)):
            for j in range(len(board)):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_O
                    value = evaluate_state(board, PLAYER_X, win_patterns, alpha, beta)
                    best_value = min(best_value, value)
                    board[i][j] = EMPTY
                    beta = min(beta, best_value)
                    if beta <= alpha:  # Alpha cut-off
                        break
        memo[board_tuple] = best_value
        return best_value

In [68]:
def get_optimal_move(board, player, win_patterns):
    """Returns the best move for the current player (X or O) using the evaluation function."""
    best_move = None
    if player == PLAYER_X:
        best_value = -float('inf')
    else:
        best_value = float('inf')

    for i in range(len(board)):
        for j in range(len(board)):
            if board[i][j] == EMPTY:
                board[i][j] = player
                value = evaluate_state(board, PLAYER_O if player == PLAYER_X else PLAYER_X, win_patterns)
                board[i][j] = EMPTY
                if player == PLAYER_X and value > best_value:
                    best_value = value
                    best_move = (i, j)
                elif player == PLAYER_O and value < best_value:
                    best_value = value
                    best_move = (i, j)

    return best_move


In [69]:
def simulate_game(board_size, win_patterns):
    """Simulates a single game of Tic-Tac-Toe with optimal moves."""
    board = [[EMPTY for _ in range(board_size)] for _ in range(board_size)]
    current_player = PLAYER_X

    while True:
        if board_full(board):
            return 0  # Draw

        move = get_optimal_move(board, current_player, win_patterns)
        if move is None:  # No available moves
            return 0  # Draw

        i, j = move
        board[i][j] = current_player

        # Check for win after the move
        winner = check_win(board, win_patterns)
        if winner is not None:
            return winner

        # Switch player
        current_player = PLAYER_O if current_player == PLAYER_X else PLAYER_X


In [75]:
def plot_win_rate(board_size, num_games=100):
    """Simulates multiple games and plots the win rate of Player X."""
    win_patterns = generate_win_patterns(board_size)
    x_wins = 0
    o_wins = 0
    draws = 0

    for _ in range(num_games):
        result = simulate_game(board_size, win_patterns)
        if result == PLAYER_X:
            x_wins += 1
        elif result == PLAYER_O:
            o_wins += 1
        else:
            draws += 1

    # Calculate rates
    x_win_rate = x_wins / num_games
    o_win_rate = o_wins / num_games
    draw_rate = draws / num_games

    # Plot the win rates
    labels = ['Player X Wins', 'Player O Wins', 'Draws']
    rates = [x_win_rate, o_win_rate, draw_rate]

    plt.bar(labels, rates, color=['blue', 'red', 'gray'])
    plt.xlabel('Outcome')
    plt.ylabel('Rate')
    plt.title(f'Win Rate for Tic-Tac-Toe {board_size}x{board_size}')
    plt.show()

In [77]:
plot_win_rate(4)